# Recomendações e Review Sprint 1 - Economiza+ MVP

**Objetivo:** Analisar gastos e revisar sprint

## Checklist:
- [ ] Gastos médios por cluster
- [ ] Top 3 categorias para economia
- [ ] Sprint1_Review.md

**Entregável:** Review + prep Sprint 2

## 1. Setup

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Carregar dados
transacoes = pd.read_csv('../data/raw/transacoes.csv')
usuarios_cluster = pd.read_csv('../data/processed/usuarios_clustered.csv')

print(f'Transações: {len(transacoes)}')
print(f'Usuários: {len(usuarios_cluster)}')
print(f'Categorias: {transacoes["categoria"].nunique()}')

Transações: 194231
Usuários: 500
Categorias: 13


In [2]:
# Juntar transações com clusters
df = transacoes.merge(usuarios_cluster[['user_id', 'cluster']], on='user_id')

# Nomear clusters (mesma lógica do notebook 05)
medias_cluster = usuarios_cluster.groupby('cluster')['taxa_poupanca'].mean()

def nomear_cluster(cluster_id):
    taxa = medias_cluster[cluster_id]
    
    if taxa > 0.15:
        return 'Poupadores'
    elif taxa > 0 and taxa <= 0.15:
        return 'Equilibrados'
    elif taxa >= -0.25:  # Entre 0% e -25%
        return 'Em Alerta'
    elif taxa >= -0.65:  # Entre -25% e -65%
        return 'Endividados Moderados'
    else:  # Abaixo de -65%
        return 'Endividados Severos'

cluster_names = {c: nomear_cluster(c) for c in usuarios_cluster['cluster'].unique()}
df['cluster_nome'] = df['cluster'].map(cluster_names)
usuarios_cluster['cluster_nome'] = usuarios_cluster['cluster'].map(cluster_names)

print('Clusters identificados:')
for cluster_id in sorted(cluster_names.keys()):
    nome = cluster_names[cluster_id]
    taxa = medias_cluster[cluster_id] * 100
    qtd = len(usuarios_cluster[usuarios_cluster['cluster'] == cluster_id])
    print(f'  Cluster {cluster_id}: {nome:25} | {qtd:3} usuários | Taxa: {taxa:+.1f}%')

Clusters identificados:
  Cluster 0: Endividados Severos       |  59 usuários | Taxa: -88.6%
  Cluster 1: Em Alerta                 | 196 usuários | Taxa: -14.8%
  Cluster 2: Endividados Moderados     | 167 usuários | Taxa: -57.7%
  Cluster 3: Poupadores                |  78 usuários | Taxa: +25.4%


## 2. Gastos Médios por Cluster

In [3]:
# Gasto MENSAL médio por usuário, agrupado por cluster e categoria
# 1. Somar gastos por usuario/categoria (total dos 6 meses)
# 2. Dividir por número de meses
# 3. Calcular média por cluster

n_meses = df['mes'].nunique()

gasto_usuario_cat = df.groupby(['user_id', 'cluster_nome', 'categoria'])['valor'].sum().reset_index()
gasto_usuario_cat['valor_mensal'] = gasto_usuario_cat['valor'] / n_meses

gastos_cluster = gasto_usuario_cat.groupby(['cluster_nome', 'categoria'])['valor_mensal'].mean().unstack(fill_value=0)

print(f'GASTO MENSAL MÉDIO POR USUÁRIO (R$) - base: {n_meses} meses')
print('='*80)
gastos_cluster.round(2)

GASTO MENSAL MÉDIO POR USUÁRIO (R$) - base: 5 meses


categoria,Alimentacao_Casa,Alimentacao_Fora,Educacao,Habitacao_Aluguel,Habitacao_Contas,Higiene_Limpeza,Lazer,Outros,Renda,Saude,Telecomunicacoes,Transporte,Vestuario
cluster_nome,,,,,,,,,,,,,
Em Alerta,673.04,306.75,187.11,653.22,448.00,114.80,113.96,75.36,3827.09,267.59,115.28,359.57,154.14
Endividados Moderados,974.76,432.53,258.56,887.82,620.84,159.82,167.86,103.98,3836.17,365.78,158.67,562.13,207.73
Endividados Severos,1357.74,619.91,381.36,1540.41,919.11,235.83,232.53,155.41,4575.24,555.73,229.91,775.61,294.00
Poupadores,1027.50,472.94,287.32,948.24,682.68,170.35,170.53,116.82,8009.04,401.48,181.05,564.59,212.20


In [4]:
# Gasto mensal médio por categoria (por usuário)
gasto_total_usuario = df.groupby(['user_id', 'categoria'])['valor'].sum() / n_meses
gasto_medio_cat = gasto_total_usuario.groupby('categoria').mean().sort_values(ascending=False)

print('GASTO MENSAL MÉDIO POR CATEGORIA (por usuário):')
print('='*50)
for cat, valor in gasto_medio_cat.items():
    print(f'{cat:25} R$ {valor:,.2f}')

GASTO MENSAL MÉDIO POR CATEGORIA (por usuário):
Renda                     R$ 4,570.79
Alimentacao_Casa          R$ 909.91
Habitacao_Aluguel         R$ 882.29
Habitacao_Contas          R$ 597.93
Transporte                R$ 508.30
Alimentacao_Fora          R$ 411.64
Saude                     R$ 355.27
Educacao                  R$ 249.53
Vestuario                 R$ 197.60
Lazer                     R$ 154.78
Telecomunicacoes          R$ 153.56
Higiene_Limpeza           R$ 152.79
Outros                    R$ 100.84


## 3. Top 3 Categorias para Economia

In [5]:
# Top 3 categorias NÃO essenciais - gasto mensal médio por usuário
df_nao_essencial = df[df['is_essencial'] == False]

gasto_ne_usuario = df_nao_essencial.groupby(['user_id', 'categoria'])['valor'].sum() / n_meses
gasto_ne_medio = gasto_ne_usuario.groupby('categoria').mean().sort_values(ascending=False)

print('TOP 3 CATEGORIAS PARA ECONOMIA (não essenciais):')
print('='*50)
print('(gasto mensal médio por usuário)')
print()
top3_economia = gasto_ne_medio.head(3)
for i, (cat, valor) in enumerate(top3_economia.items(), 1):
    print(f'{i}. {cat:20} R$ {valor:,.2f}/mês')

TOP 3 CATEGORIAS PARA ECONOMIA (não essenciais):
(gasto mensal médio por usuário)

1. Alimentacao_Fora     R$ 411.64/mês
2. Vestuario            R$ 197.60/mês
3. Lazer                R$ 154.78/mês


In [6]:
# Oportunidades de economia por cluster (gasto mensal médio por usuário)
print('OPORTUNIDADES DE ECONOMIA POR CLUSTER:')
print('='*60)
print('(gasto mensal médio por usuário)')

recomendacoes_cluster = {}

for cluster_nome in sorted(df['cluster_nome'].unique()):
    df_cluster = df[(df['cluster_nome'] == cluster_nome) & (df['is_essencial'] == False)]
    
    # Gasto mensal médio por usuário do cluster
    gasto_cluster_usuario = df_cluster.groupby(['user_id', 'categoria'])['valor'].sum() / n_meses
    gasto_cluster_medio = gasto_cluster_usuario.groupby('categoria').mean().sort_values(ascending=False)
    
    top_cat = gasto_cluster_medio.head(3)
    recomendacoes_cluster[cluster_nome] = top_cat.index.tolist()
    
    print(f'\n{cluster_nome.upper()}:')
    for i, (cat, val) in enumerate(top_cat.items(), 1):
        print(f'  {i}. Reduzir {cat}: R$ {val:,.2f}/mês')

OPORTUNIDADES DE ECONOMIA POR CLUSTER:
(gasto mensal médio por usuário)

EM ALERTA:
  1. Reduzir Alimentacao_Fora: R$ 306.75/mês
  2. Reduzir Vestuario: R$ 154.14/mês
  3. Reduzir Lazer: R$ 113.96/mês

ENDIVIDADOS MODERADOS:
  1. Reduzir Alimentacao_Fora: R$ 432.53/mês
  2. Reduzir Vestuario: R$ 207.73/mês
  3. Reduzir Lazer: R$ 167.86/mês

ENDIVIDADOS SEVEROS:
  1. Reduzir Alimentacao_Fora: R$ 619.91/mês
  2. Reduzir Vestuario: R$ 294.00/mês
  3. Reduzir Lazer: R$ 232.53/mês

POUPADORES:
  1. Reduzir Alimentacao_Fora: R$ 472.94/mês
  2. Reduzir Vestuario: R$ 212.20/mês
  3. Reduzir Lazer: R$ 170.53/mês


## 4. Gerar Sprint1_Review.md

In [7]:
# Métricas do Sprint 1
total_usuarios = len(usuarios_cluster)
total_clusters = usuarios_cluster['cluster'].nunique()
usuarios_risco = len(usuarios_cluster[usuarios_cluster['taxa_poupanca'] < 0])
taxa_media = usuarios_cluster['taxa_poupanca'].mean() * 100

# Métricas de validação H2
silhouette_score = 0.2568
davies_bouldin = 1.1959
silhouette_target = 0.5
db_target = 1.0

# Gerar markdown
review_md = f"""# Sprint 1 Review - Economiza+ MVP

## Resumo

| Metrica | Valor |
|---------|-------|
| Usuarios analisados | {total_usuarios} |
| Clusters identificados | {total_clusters} |
| Usuarios em risco | {usuarios_risco} ({usuarios_risco/total_usuarios*100:.1f}%) |
| Taxa poupanca media | {taxa_media:.1f}% |

## Validacao da Hipotese H2

**H2: Deteccao de Padroes** - Algoritmos de clustering podem identificar padroes de gastos com precisao superior a 80%.

| Metrica | Target | Resultado | Status |
|---------|--------|-----------|--------|
| Silhouette Score | > {silhouette_target} | {silhouette_score:.4f} | NAO ATINGIDO |
| Davies-Bouldin Index | < {db_target} | {davies_bouldin:.4f} | NAO ATINGIDO |
| Clusters interpretaveis | Sim | Sim | ATINGIDO |

### Analise

- **Silhouette Score ({silhouette_score:.2f})**: Abaixo do target ({silhouette_target}), indicando sobreposicao entre clusters
- **Davies-Bouldin ({davies_bouldin:.2f})**: Acima do target ({db_target}), confirmando clusters nao bem separados
- **Interpretabilidade**: Apesar das metricas, os 4 clusters sao claramente interpretaveis com perfis distintos de risco financeiro

### Decisao

Para o MVP, **aceitamos os resultados** pois:
1. Os clusters tem significado de negocio claro
2. Permitem recomendacoes personalizadas por perfil e nivel de risco
3. Melhorias podem ser feitas no Sprint 2 (ajuste de features, remocao de outliers)

## Entregas Concluidas

- [x] EDA Basico
- [x] Feature Engineering (5 features)
- [x] Clustering K-means (K=4)
- [x] Validacao do Modelo
- [x] Interpretacao dos Clusters
- [x] Analise de Recomendacoes

## Perfis Identificados

"""

# Ordenar por taxa de poupança (dos piores para os melhores)
perfis_ordenados = usuarios_cluster.groupby(['cluster', 'cluster_nome'])['taxa_poupanca'].mean().sort_values()

for (cluster_id, nome), taxa in perfis_ordenados.items():
    qtd = len(usuarios_cluster[usuarios_cluster['cluster'] == cluster_id])
    review_md += f"- **{nome}**: {qtd} usuarios ({taxa*100:+.1f}% poupanca)\n"

review_md += """
## Top 3 Categorias para Economia

(gasto mensal medio por usuario - nao essenciais)

"""

for i, (cat, valor) in enumerate(top3_economia.items(), 1):
    review_md += f"{i}. **{cat}** - R$ {valor:,.2f}/mes\n"

review_md += """
## Preparacao Sprint 2

### Proximos Passos

1. Implementar sistema de recomendacoes por perfil (H1)
2. Treinar detector de anomalias Isolation Forest (H6)
3. Integrar clustering + recomendacao + anomalias
4. Criar notebook de demonstracao

### Melhorias Identificadas para Clustering

- Testar remocao de outliers antes do clustering
- Considerar adicionar features comportamentais
- Avaliar normalizacao alternativa (MinMaxScaler)
- Refinar limites entre categorias de risco

---
*Sprint 1 concluido em Janeiro/2026*
"""

# Salvar arquivo
with open('../outputs/Sprint1_Review.md', 'w', encoding='utf-8') as f:
    f.write(review_md)

print('Entregavel salvo: ../outputs/Sprint1_Review.md')
print('\n' + '='*50)
print('PREVIEW:')
print('='*50)
print(review_md)

Entregavel salvo: ../outputs/Sprint1_Review.md

PREVIEW:
# Sprint 1 Review - Economiza+ MVP

## Resumo

| Metrica | Valor |
|---------|-------|
| Usuarios analisados | 500 |
| Clusters identificados | 4 |
| Usuarios em risco | 386 (77.2%) |
| Taxa poupanca media | -31.6% |

## Validacao da Hipotese H2

**H2: Deteccao de Padroes** - Algoritmos de clustering podem identificar padroes de gastos com precisao superior a 80%.

| Metrica | Target | Resultado | Status |
|---------|--------|-----------|--------|
| Silhouette Score | > 0.5 | 0.2568 | NAO ATINGIDO |
| Davies-Bouldin Index | < 1.0 | 1.1959 | NAO ATINGIDO |
| Clusters interpretaveis | Sim | Sim | ATINGIDO |

### Analise

- **Silhouette Score (0.26)**: Abaixo do target (0.5), indicando sobreposicao entre clusters
- **Davies-Bouldin (1.20)**: Acima do target (1.0), confirmando clusters nao bem separados
- **Interpretabilidade**: Apesar das metricas, os 4 clusters sao claramente interpretaveis com perfis distintos de risco financeiro


## 5. Checklist Final

In [8]:
print('='*50)
print('CHECKLIST - RECOMENDAÇÕES E REVIEW')
print('='*50)
print('[x] Gastos médios por cluster')
print('[x] Top 3 categorias para economia')
print('[x] Sprint1_Review.md')
print('='*50)
print('\nEntregável: ../outputs/Sprint1_Review.md')
print('\nSPRINT 1 CONCLUÍDO!')

CHECKLIST - RECOMENDAÇÕES E REVIEW
[x] Gastos médios por cluster
[x] Top 3 categorias para economia
[x] Sprint1_Review.md

Entregável: ../outputs/Sprint1_Review.md

SPRINT 1 CONCLUÍDO!
